## Carga e Ingeniería de datos (ACTUALIZADA)

In [1]:
import pandas as pd

import sys
import os

# Add the root 'code' directory to sys.path so Python knows where to find 'common'
sys.path.append(os.path.abspath(os.path.join('..')))  # Adjust as needed

import df_dist_source
import common.source as src
 

data_path = '../../data'

tables_path = f"{data_path}/data_raw/"

address_df = pd.read_csv(tables_path + "address.csv")
cities_df = pd.read_csv(tables_path + "cities.csv")
clients_df = pd.read_csv(tables_path + "clients.csv")
service_labors_df = pd.read_csv(tables_path + "service_labors.csv")
services_df = pd.read_csv(tables_path + "services.csv")


In [2]:
import csv, pandas as pd

path = tables_path + "addresses_202507071526.csv"
with open(path, encoding="ISO-8859-1") as f:
    sample = f.read(2048)
    dialect = csv.Sniffer().sniff(sample)
    print("delimitador =", dialect.delimiter,
          "quotechar =", dialect.quotechar)

address_df = pd.read_csv(
    path,
    encoding="ISO-8859-1",
    sep=dialect.delimiter,
    quotechar=dialect.quotechar,
    engine="python",
    on_bad_lines="warn"      # avisa si hay líneas problemáticas
)

import pandas as pd

address_df = pd.read_csv(
    tables_path + "addresses_202507071526.csv",
    encoding="ISO-8859-1",
    engine="python",
    on_bad_lines="skip"    # descarta las líneas mal formateadas
)


delimitador = , quotechar = "


### datetime types

In [3]:
# 1) Convertir en service_labors_df
for col in ['labor_created_at', 'labor_start_date', 'labor_end_date']:
    service_labors_df[col] = pd.to_datetime(
        service_labors_df[col],
        format="%Y-%m-%d %H:%M:%S.%f %z",
        utc=True
    ).dt.tz_convert('America/Bogota')

# 2) Convertir en services_df
for col in ['created_at', 'schedule_date']:
    services_df[col] = pd.to_datetime(
        services_df[col],
        format="%Y-%m-%d %H:%M:%S.%f %z",
        utc=True
    ).dt.tz_convert('America/Bogota')

# 3) Verificar tipos de las columnas
print(service_labors_df[['labor_created_at','labor_start_date','labor_end_date']].dtypes)
print(services_df[['created_at','schedule_date']].dtypes)

labor_created_at    datetime64[ns, America/Bogota]
labor_start_date    datetime64[ns, America/Bogota]
labor_end_date      datetime64[ns, America/Bogota]
dtype: object
created_at       datetime64[ns, America/Bogota]
schedule_date    datetime64[ns, America/Bogota]
dtype: object


In [4]:
address_df['address_id'] = address_df['id']

# 1) Prepara la mini‐tabla de lookup únicamente por shop
address_shop = (
    address_df
    .dropna(subset=["shop"])
    .drop_duplicates(subset=["shop"])
    .set_index("shop")[["address_id", "point", "name"]]
)

# 2) Merge labors con services (igual que antes)
unified = (
    service_labors_df
    .merge(services_df, on="service_id", how="left")
)

# 3) Mapea address_id solo desde shop
unified["address_id"] = (
    unified["shop"]
    .map(address_shop["address_id"])
    .astype("Int64")           # entero nullable
)

# 4) Mapea address_point solo desde shop
unified["address_point"] = unified["shop"].map(address_shop["point"])

# 5) Mapea address_name solo desde shop
unified["address_name"] = unified["shop"].map(address_shop["name"])

# 6) Comprueba el resultado
display(unified)


,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,state_service,start_address_id,start_address_point,end_address_id,end_address_point,city,service_mode,address_id,address_point,address_name
0,64839,110329.0,3.0,Wash and Polish,WASH_AND_POLISH,119900.0,2022-10-28 11:04:58.814000-05:00,NaT,NaT,NaN,...,CANCELED,40184.0,POINT (-74.1229968 4.6082095),40184.0,POINT (-74.1229968 4.6082095),149.0,WITH_DRIVER,4885,POINT (-74.07294885404518 4.650544279518066),Polish Car Calle 63
1,64839,112351.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,0.0,2022-11-01 17:34:05.254000-05:00,NaT,NaT,8594.0,...,CANCELED,40184.0,POINT (-74.1229968 4.6082095),40184.0,POINT (-74.1229968 4.6082095),149.0,WITH_DRIVER,<NA>,NaN,NaN
2,64839,133512.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,0.0,2022-12-15 08:53:50.909000-05:00,NaT,NaT,8594.0,...,CANCELED,40184.0,POINT (-74.1229968 4.6082095),40184.0,POINT (-74.1229968 4.6082095),149.0,WITH_DRIVER,<NA>,NaN,NaN
3,72389,126066.0,6.0,General Mechanics,GENERAL_MECHANICS,60000.0,2022-11-29 08:21:09.247000-05:00,2023-01-13 07:00:00-05:00,2023-01-13 07:00:00-05:00,NaN,...,COMPLETED,34642.0,POINT (-74.0618875 4.663559499999999),34642.0,POINT (-74.0618875 4.663559499999999),149.0,WITH_DRIVER,30263,POINT (-74.0730511 4.6995007),BONPARTE 116
4,72807,126912.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,35000.0,2022-11-30 11:18:02.219000-05:00,2023-01-14 10:04:00-05:00,2023-01-14 10:48:00-05:00,2306.0,...,COMPLETED,38667.0,POINT (-76.51785562935675 3.475724296400637),38667.0,POINT (-76.51785562935675 3.475724296400637),1004.0,WITH_DRIVER,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200430,246621,342204.0,3.0,Wash and Polish,WASH_AND_POLISH,29640.0,2025-05-26 12:02:19.759000-05:00,NaT,NaT,NaN,...,REQUESTED,118836.0,POINT (-74.0487971 4.728055500000001),118836.0,POINT (-74.0487971 4.728055500000001),149.0,WITHOUT_DRIVER,30243,POINT (-74.0494164 4.729390599999999),CONTINENTAL CAR WASH
200431,246622,342205.0,3.0,Wash and Polish,WASH_AND_POLISH,26600.0,2025-05-26 12:05:24.429000-05:00,2025-05-26 12:06:47.759000-05:00,NaT,NaN,...,IN_PROGRESS,135267.0,POINT (-74.069678 4.7194277),135267.0,POINT (-74.069678 4.7194277),149.0,WITHOUT_DRIVER,67252,POINT (-74.07305460423231 4.713989646958405),Agility Car Wash
200432,246623,342206.0,788.0,Inspección preoperativa,OTHER_LABORS,0.0,2025-05-26 12:05:35.960000-05:00,2025-05-26 12:05:37.297000-05:00,NaT,NaN,...,COMPLETED,NaN,NaN,NaN,NaN,1004.0,WITHOUT_DRIVER,113486,POINT (-76.5277114 3.3710199),Cedi Cali
200433,246624,342207.0,6.0,General Mechanics,GENERAL_MECHANICS,NaN,2025-05-26 12:05:42.753000-05:00,NaT,NaT,NaN,...,IN_PROGRESS,118304.0,POINT (-75.670001 4.805639999999999),118304.0,POINT (-75.670001 4.805639999999999),830.0,WITHOUT_DRIVER,118304,POINT (-75.670001 4.805639999999999),Roble Motor


## Directorio de Alfreds

In [8]:
directorio_df = pd.read_csv(f"{data_path}/data_pre/directorio_df.csv")
# directorio_df

### Distancias

In [9]:
# Crear df_dist solo con las columnas de punto
df_dist = unified.copy()

# Calcular las tres distancias
df_dist['start_to_point'] = df_dist.apply(
    lambda row: df_dist_source.fn_dir.haversine_distance(row['start_address_point'], row['address_point']),
    axis=1
)
df_dist['point_to_end'] = df_dist.apply(
    lambda row: df_dist_source.fn_dir.haversine_distance(row['address_point'], row['end_address_point']),
    axis=1
)
df_dist['start_to_end'] = df_dist.apply(
    lambda row: df_dist_source.fn_dir.haversine_distance(row['start_address_point'], row['end_address_point']),
    axis=1
)

# Mostrar resultado
df_dist

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,end_address_id,end_address_point,city,service_mode,address_id,address_point,address_name,start_to_point,point_to_end,start_to_end
0,64839,110329.0,3.0,Wash and Polish,WASH_AND_POLISH,119900.0,2022-10-28 11:04:58.814000-05:00,NaT,NaT,NaN,...,40184.0,POINT (-74.1229968 4.6082095),149.0,WITH_DRIVER,4885,POINT (-74.07294885404518 4.650544279518066),Polish Car Calle 63,7.275169,7.275169,0.0
1,64839,112351.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,0.0,2022-11-01 17:34:05.254000-05:00,NaT,NaT,8594.0,...,40184.0,POINT (-74.1229968 4.6082095),149.0,WITH_DRIVER,<NA>,NaN,NaN,<NA>,<NA>,0.0
2,64839,133512.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,0.0,2022-12-15 08:53:50.909000-05:00,NaT,NaT,8594.0,...,40184.0,POINT (-74.1229968 4.6082095),149.0,WITH_DRIVER,<NA>,NaN,NaN,<NA>,<NA>,0.0
3,72389,126066.0,6.0,General Mechanics,GENERAL_MECHANICS,60000.0,2022-11-29 08:21:09.247000-05:00,2023-01-13 07:00:00-05:00,2023-01-13 07:00:00-05:00,NaN,...,34642.0,POINT (-74.0618875 4.663559499999999),149.0,WITH_DRIVER,30263,POINT (-74.0730511 4.6995007),BONPARTE 116,4.183598,4.183598,0.0
4,72807,126912.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,35000.0,2022-11-30 11:18:02.219000-05:00,2023-01-14 10:04:00-05:00,2023-01-14 10:48:00-05:00,2306.0,...,38667.0,POINT (-76.51785562935675 3.475724296400637),1004.0,WITH_DRIVER,<NA>,NaN,NaN,<NA>,<NA>,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200430,246621,342204.0,3.0,Wash and Polish,WASH_AND_POLISH,29640.0,2025-05-26 12:02:19.759000-05:00,NaT,NaT,NaN,...,118836.0,POINT (-74.0487971 4.728055500000001),149.0,WITHOUT_DRIVER,30243,POINT (-74.0494164 4.729390599999999),CONTINENTAL CAR WASH,0.163552,0.163552,0.0
200431,246622,342205.0,3.0,Wash and Polish,WASH_AND_POLISH,26600.0,2025-05-26 12:05:24.429000-05:00,2025-05-26 12:06:47.759000-05:00,NaT,NaN,...,135267.0,POINT (-74.069678 4.7194277),149.0,WITHOUT_DRIVER,67252,POINT (-74.07305460423231 4.713989646958405),Agility Car Wash,0.711098,0.711098,0.0
200432,246623,342206.0,788.0,Inspección preoperativa,OTHER_LABORS,0.0,2025-05-26 12:05:35.960000-05:00,2025-05-26 12:05:37.297000-05:00,NaT,NaN,...,NaN,NaN,1004.0,WITHOUT_DRIVER,113486,POINT (-76.5277114 3.3710199),Cedi Cali,<NA>,<NA>,<NA>
200433,246624,342207.0,6.0,General Mechanics,GENERAL_MECHANICS,NaN,2025-05-26 12:05:42.753000-05:00,NaT,NaT,NaN,...,118304.0,POINT (-75.670001 4.805639999999999),830.0,WITHOUT_DRIVER,118304,POINT (-75.670001 4.805639999999999),Roble Motor,0.0,0.0,0.0


In [10]:
# 1) Descartar labors sin fechas
df_temp = df_dist.dropna(subset=['labor_start_date', 'labor_end_date']).copy()

# 2) Calcular duración como timedelta
df_temp['duration_td'] = df_temp['labor_end_date'] - df_temp['labor_start_date']

# 3) Filtrar solo duraciones ≤ 1 día
df_temp = df_temp[df_temp['duration_td'] <= pd.Timedelta(days=1)]

# 4) Convertir a minutos
df_temp['duration_min'] = df_temp['duration_td'].dt.total_seconds() / 60

# 5) Agrupar y promediar
avg_times = (
    df_temp
    .groupby('labor_name')['duration_min']
    .mean()
    .reset_index()
    .rename(columns={'duration_min': 'avg_duration_min'})
)

avg_times


,labor_name,avg_duration_min
0,Aceite promocional,10.000000
1,Administración de Flotas,11.789474
2,Alfred Favor,20.000000
3,Alfred Initial Transport,105.107405
4,Alfred Initial Transport Runt,138.250000
...,...,...
88,Toma de Improntas,240.140789
89,Trailer transport,34.371876
90,Traspaso,70.714286
91,Vehiculo Sustituto,3.142857


In [11]:
# 1) Asegurarnos de tener fechas de inicio válidas
df_valid = df_dist.dropna(subset=['labor_start_date']).copy()

# 2) Extraer sólo la fecha (sin hora)
df_valid['day'] = df_valid['labor_start_date'].dt.date

# 3) Contar cuántas labores hay por día
labors_per_day = (
    df_valid
      .groupby('day')
      .size()
      .sort_values(ascending=False)
      .rename('labors_count')
      .reset_index()
)

# 4) Mostrar los días con más labores (por ejemplo, top 10)
print(labors_per_day.head(10))


          day  labors_count
0  2024-10-03           574
1  2024-12-03           562
2  2024-10-23           557
3  2024-09-23           556
4  2024-10-07           555
5  2024-09-25           554
6  2024-12-05           549
7  2024-10-24           547
8  2024-10-08           545
9  2024-11-25           542


### Funciones de filtrado

In [12]:
df_bogota = src.filtros.filter_labors_by_city(df_dist, 149)
df_bogota

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,end_address_id,end_address_point,city,service_mode,address_id,address_point,address_name,start_to_point,point_to_end,start_to_end
0,64839,110329.0,3.0,Wash and Polish,WASH_AND_POLISH,119900.0,2022-10-28 11:04:58.814000-05:00,NaT,NaT,NaN,...,40184.0,POINT (-74.1229968 4.6082095),149.0,WITH_DRIVER,4885,POINT (-74.07294885404518 4.650544279518066),Polish Car Calle 63,7.275169,7.275169,0.0
1,64839,112351.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,0.0,2022-11-01 17:34:05.254000-05:00,NaT,NaT,8594.0,...,40184.0,POINT (-74.1229968 4.6082095),149.0,WITH_DRIVER,<NA>,NaN,NaN,<NA>,<NA>,0.0
2,64839,133512.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,0.0,2022-12-15 08:53:50.909000-05:00,NaT,NaT,8594.0,...,40184.0,POINT (-74.1229968 4.6082095),149.0,WITH_DRIVER,<NA>,NaN,NaN,<NA>,<NA>,0.0
3,72389,126066.0,6.0,General Mechanics,GENERAL_MECHANICS,60000.0,2022-11-29 08:21:09.247000-05:00,2023-01-13 07:00:00-05:00,2023-01-13 07:00:00-05:00,NaN,...,34642.0,POINT (-74.0618875 4.663559499999999),149.0,WITH_DRIVER,30263,POINT (-74.0730511 4.6995007),BONPARTE 116,4.183598,4.183598,0.0
9,73987,129333.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,35000.0,2022-12-05 12:02:06.160000-05:00,2023-01-05 07:27:00-05:00,2023-01-05 08:33:00-05:00,14662.0,...,33182.0,POINT (-74.0752624 4.7271946),149.0,WITH_DRIVER,<NA>,NaN,NaN,<NA>,<NA>,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200425,246617,342198.0,4.0,Tecnomecanica,SAFETI_INSPECTION,313692.0,2025-05-26 11:54:37.454000-05:00,NaT,NaT,NaN,...,137553.0,POINT (-74.1143538 4.668493),149.0,WITH_DRIVER,17342,POINT (-74.1233701 4.642520299999999),Automas Calle 13,3.056019,3.056019,0.0
200426,246617,342197.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,35000.0,2025-05-26 11:54:37.416000-05:00,NaT,NaT,NaN,...,137553.0,POINT (-74.1143538 4.668493),149.0,WITH_DRIVER,<NA>,NaN,NaN,<NA>,<NA>,0.0
200427,246618,342200.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,80663.0,2025-05-26 11:54:54.647000-05:00,NaT,NaT,NaN,...,29156.0,POINT (-74.03727236805345 4.878226952715764),149.0,WITH_DRIVER,<NA>,NaN,NaN,<NA>,<NA>,4.399798
200430,246621,342204.0,3.0,Wash and Polish,WASH_AND_POLISH,29640.0,2025-05-26 12:02:19.759000-05:00,NaT,NaT,NaN,...,118836.0,POINT (-74.0487971 4.728055500000001),149.0,WITHOUT_DRIVER,30243,POINT (-74.0494164 4.729390599999999),CONTINENTAL CAR WASH,0.163552,0.163552,0.0


# Guardar en pickle

In [13]:
# # ==============================================
# # 1) Generar el archivo de distancias con 'manhattan'
# # ==============================================
# import pickle
# import pandas as pd
# import math
# from itertools import combinations

# # ——————————————————————————————
# # Reuso tu función `parse_point` y `distance`
# # ——————————————————————————————
# def parse_point(point_str):
#     if pd.isna(point_str):
#         return None, None
#     s = point_str.strip()
#     if not s.startswith("POINT"):
#         return None, None
#     lon, lat = map(float, s.lstrip('POINT').strip(' ()').split())
#     return lat, lon

# OSRM_URL = "http://router.project-osrm.org/route/v1/driving/"
# def distance(p1, p2, method='manhattan', timeout=5):
#     lat1, lon1 = parse_point(p1)
#     lat2, lon2 = parse_point(p2)
#     if None in (lat1, lon1, lat2, lon2):
#         return float('nan')
#     if method == 'haversine':
#         φ1, φ2 = map(math.radians, (lat1, lat2))
#         dφ = math.radians(lat2 - lat1)
#         dλ = math.radians(lon2 - lon1)
#         a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
#         return 2 * 6371 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
#     if method == 'osrm':
#         coords = f"{lon1},{lat1};{lon2},{lat2}"
#         try:
#             import requests
#             r = requests.get(OSRM_URL + coords + "?overview=false", timeout=timeout)
#             r.raise_for_status()
#             return r.json()['routes'][0]['distance'] / 1000
#         except:
#             return distance(p1, p2, method='haversine')
#     # Manhattan aproximada
#     KM_PER_DEG_LAT = 111.32
#     mean_lat = math.radians((lat1 + lat2) / 2)
#     dlat = abs(lat1 - lat2) * KM_PER_DEG_LAT
#     dlon = abs(lon1 - lon2) * KM_PER_DEG_LAT * math.cos(mean_lat)
#     return dlat + dlon

# # ——————————————————————————————
# # Recolectar todos los puntos
# # ——————————————————————————————
# puntos = set()

# # posiciones iniciales de conductores en Bogotá
# for _, d in directorio_df[directorio_df['city']=='BOGOTA'].iterrows():
#     if pd.isna(d['latitud']) or pd.isna(d['longitud']):
#         continue
#     puntos.add(f"POINT ({d['longitud']} {d['latitud']})")

# # puntos de los labors
# for col in ['address_point', 'start_address_point', 'end_address_point']:
#     pts = df_cleaned[col].dropna().unique()
#     puntos.update(pts)

# puntos = list(puntos)

# # ——————————————————————————————
# # Calcular distancias con método 'manhattan'
# # ——————————————————————————————
# dist_dict = {}
# for p1, p2 in combinations(puntos, 2):
#     d = distance(p1, p2, method='osrm')
#     dist_dict[(p1, p2)] = d
#     dist_dict[(p2, p1)] = d
# for p in puntos:
#     dist_dict[(p, p)] = 0.0

# # ——————————————————————————————
# # Guardar en pickle
# # ——————————————————————————————
# with open('distances.pkl', 'wb') as f:
#     pickle.dump(dist_dict, f)

# print(f"✔ Se calcularon distancias para {len(puntos)} puntos y se guardó 'distances.pkl'")


In [14]:
df_dist.to_pickle(f'{data_path}/data_clean/df_dist.pkl')
df_dist.to_csv(f'{data_path}/data_clean/df_dist.csv', index=False)

Numero de labores por día

In [15]:
# import os
# import pickle
# import pandas as pd
# import math
# import requests
# import random
# from datetime import timedelta
# from filtros_df import filter_labors_by_city

# # ——————————————————————————————
# # Parámetros y cargas base
# # ——————————————————————————————
# SEED = 10
# random.seed(SEED)

# directorio_df = pd.read_csv("Datos/Procesados/directorio_df.csv")
# df_dist        = pd.read_pickle("df_dist.pkl")
# with open("distances.pkl", "rb") as f:
#     DIST_DICT = pickle.load(f)

# def parse_point(s):
#     if pd.isna(s) or not s.strip().startswith("POINT"):
#         return None, None
#     lon, lat = map(float, s.lstrip("POINT").strip(" ()").split())
#     return lat, lon

# OSRM_URL = "http://router.project-osrm.org/route/v1/driving/"
# def distance(p1, p2, method="haversine", timeout=5):
#     if method=="precalced":
#         return DIST_DICT.get((p1,p2), float("nan"))
#     lat1, lon1 = parse_point(p1)
#     lat2, lon2 = parse_point(p2)
#     if None in (lat1,lon1,lat2,lon2):
#         return float("nan")
#     if method=="haversine":
#         φ1, φ2 = map(math.radians,(lat1,lat2))
#         dφ = math.radians(lat2-lat1)
#         dλ = math.radians(lon2-lon1)
#         a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
#         return 2*6371*math.atan2(math.sqrt(a),math.sqrt(1-a))
#     if method=="osrm":
#         coords = f"{lon1},{lat1};{lon2},{lat2}"
#         try:
#             r = requests.get(OSRM_URL+coords+"?overview=false",timeout=timeout)
#             r.raise_for_status()
#             return r.json()['routes'][0]['distance']/1000
#         except:
#             return distance(p1,p2,method='haversine')
#     KM_PER_DEG_LAT = 111.32
#     mean_lat = math.radians((lat1+lat2)/2)
#     dlat = abs(lat1-lat2)*KM_PER_DEG_LAT
#     dlon = abs(lon1-lon2)*KM_PER_DEG_LAT*math.cos(mean_lat)
#     return dlat+dlon

# def process_group(grp):
#     # Sólo transportes
#     A_idx = list(grp.index[grp['labor_category']=='VEHICLE_TRANSPORTATION'])
#     if not A_idx:
#         return grp.iloc[0:0]
#     # Sólo shops válidos
#     B_idx = [i for i in grp.index[grp['labor_category']!='VEHICLE_TRANSPORTATION']
#              if pd.notna(grp.at[i,'shop']) and pd.notna(grp.at[i,'address_point'])]
#     if not B_idx:
#         return grp.iloc[0:0]

#     # Seleccionar primer transporte
#     inits = [i for i in A_idx if grp.at[i,'labor_name']=='Alfred Initial Transport']
#     firstA = inits[0] if inits else A_idx[0]
#     A_remain = [i for i in A_idx if i!=firstA]

#     # Ordenar shops por distancia al punto inicial
#     start_pt = grp['start_address_point'].iloc[0]
#     dist_map = {
#         i: (float('inf') if pd.isna((d:=distance(start_pt, grp.at[i,'address_point']))) else d)
#         for i in B_idx
#     }
#     max_B = len(A_idx) - 1
#     B_sorted = sorted(B_idx, key=lambda i: dist_map[i])[:max_B]

#     # A adicionales en orden de hora
#     A_remain_sorted = sorted(A_remain, key=lambda i: grp.at[i,'labor_start_date'])
#     needed_A = [firstA] + A_remain_sorted[:len(B_sorted)]

#     # Intercalar A y B
#     final = []
#     for j, b in enumerate(B_sorted):
#         final += [needed_A[j], b]
#     final.append(needed_A[-1])

#     return grp.loc[final]

# # Filtrar Bogotá
# df_bogota = filter_labors_by_city(df_dist, 149)

# # Carpeta de salida
# out_dir = "processed_by_day"
# os.makedirs(out_dir, exist_ok=True)

# # Para crear DataFrame vacío con las columnas adecuadas
# all_cols = list(df_bogota.columns) + ["map_start_point","map_end_point"]

# summary = []
# for day in sorted(df_bogota["schedule_date"].dt.date.unique()):
#     df_day = (
#         df_bogota[df_bogota["schedule_date"].dt.date == day]
#           .query("state_service!='CANCELED'")
#           .sort_values(["service_id","labor_start_date"])
#           .reset_index(drop=True)
#     )

#     # services_map_df diario
#     rows = []
#     for svc, grp in df_day.groupby("service_id", sort=False):
#         idxs = grp.index.tolist(); n = len(idxs)
#         for i, idx in enumerate(idxs):
#             r = grp.loc[idx]
#             if n==1:
#                 sp, ep = r["start_address_point"], r["end_address_point"]
#             else:
#                 if r["labor_category"]=="VEHICLE_TRANSPORTATION":
#                     sp = r["start_address_point"]
#                     ep = grp.iloc[i+1]["end_address_point"] if i<n-1 else r["end_address_point"]
#                 else:
#                     sp = ep = r["address_point"]
#             rows.append({
#                 "service_id": svc,
#                 "labor_id":   r["labor_id"],
#                 "map_start_point": sp,
#                 "map_end_point":   ep
#             })

#     services_map_df = pd.DataFrame(rows,
#         columns=["service_id","labor_id","map_start_point","map_end_point"])
#     if not services_map_df.empty:
#         services_map_df["labor_id"] = services_map_df["labor_id"].astype(df_day["labor_id"].dtype)

#     # plantillas limpias
#     cleaned = []
#     for svc, grp in df_day.groupby("service_id", sort=False):
#         pg = process_group(grp)
#         if not pg.empty:
#             cleaned.append(pg)

#     if cleaned:
#         df_cleaned = (
#             pd.concat(cleaned, ignore_index=True)
#               .merge(services_map_df, on=["service_id","labor_id"], how="left")
#         )
#     else:
#         df_cleaned = pd.DataFrame(columns=all_cols)

#     # guardar
#     fn = os.path.join(out_dir, f"cleaned_{day}.pkl")
#     df_cleaned.to_pickle(fn)
#     summary.append({"date": day, "n_labors": len(df_cleaned), "file": fn})

# # Resumen
# summary_df = pd.DataFrame(summary).sort_values("n_labors", ascending=False).reset_index(drop=True)
# print("Días con más labores:\n", summary_df[["date","n_labors"]].head())

# # opcional guardar CSV
# summary_df.to_csv(os.path.join(out_dir, "daily_summary.csv"), index=False)


In [16]:
# summary_df.to_csv(os.path.join(out_dir, "daily_summary.csv"), index=False)

In [17]:
summary_df = pd.read_csv("processed_by_day/daily_summary.csv")
summary_df

FileNotFoundError: [Errno 2] No such file or directory: 'processed_by_day/daily_summary.csv'